# Class 5: APIs

## Ways to get data

Method | How it happens | Pros | Cons
--- | :--- | :--- | :---
**Bulk** | Download, someone hands you a flash drive, etc. | Fast, one-time transfer | Can be large
**Scraping** | Data only available through a web site, PDF, or doc | You can turn anything into data | Tedious; fragile
**APIs** | If organization makes one available | Usually allows some filtering; can always pull latest-and-greatest | Requires network connection for every pull; higher barrier to entry (reading documentation); subject to availability and performance of API

## Scraping

Common tools:

- [Beautiful Soup package](https://realpython.com/beautiful-soup-web-scraper-python/)
- [pandas' `read_html()`](https://pandas.pydata.org/docs/user_guide/io.html#html)

Pull table from [Wikipedia's list of countries and dependencies by population](https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population):

In [1]:
import pandas as pd

tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population', match='Rank')
pop = tables[0]
pop

,Rank,Country / Dependency,Region,Population,Percentage of the world,Date,Source (official or from the United Nations),Notes
0,–,World,NaN,7946452000,100%,28 Apr 2022,UN projection[1],NaN
1,1,China,Asia,1412600000,NaN,31 Dec 2021,National annual estimate[2],The population figure refers to mainland China...
2,2,India,Asia,1375734450,NaN,28 Apr 2022,National population clock[3],The figure includes the population of Jammu an...
3,3,United States,Americas,332647492,NaN,28 Apr 2022,National population clock[4],The figure includes the 50 states and the Dist...
4,4,Indonesia,Asia[b],272248500,NaN,1 Jul 2021,National annual estimate[5],NaN
...,...,...,...,...,...,...,...,...
237,–,Niue (New Zealand),Oceania,1549,NaN,1 Jul 2021,National annual projection[92],NaN
238,–,Tokelau (New Zealand),Oceania,1501,NaN,1 Jul 2021,National annual projection[92],NaN
239,195,Vatican City,Europe,825,NaN,1 Feb 2019,Monthly national estimate[196],The total population of 825 consisted of 453 r...
240,–,Cocos (Keeling) Islands (Australia),Oceania,573,NaN,30 Jun 2020,National annual estimate[195],NaN


### Data is only available if it's available

## API calls in the wild

1. Go to [Candidates page on fec.gov](https://www.fec.gov/data/candidates/?has_raised_funds=true&is_active_candidate=true).
1. Right click and `Inspect`.
   - [More info about opening Developer Tools in various browsers.](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_are_browser_developer_tools#how_to_open_the_devtools_in_your_browser)
1. Go to the `Network` tab and reload.
1. Filter to `XHR`.
1. Click the API call.

### Parts of a URL

![URL structure](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL/mdn-url-all.png)

[source](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL#basics_anatomy_of_a_url)

For APIs:

- Often split into "base URL" + "endpoint"
- Anchors aren't relevant

### API documentation

[FEC API](https://api.open.fec.gov/developers/)

### Try it out

1. Visit https://www.fec.gov/data/candidates/
1. In the Network tab's request list, right-click the API call.
1. Click `Open in New Tab`.
1. Replace the API key with `DEMO_KEY`.

## API calls from Python

Usually one of two ways:

- A software development kit (SDK) like [sodapy](https://pypi.org/project/sodapy/)
   - Abstracts the details away
   - Not available for all APIs
   - May have limitations
- [The `requests` package](https://docs.python-requests.org/) (nothing to do with 311 requests)

In [2]:
import requests

params = {
    'api_key': 'DEMO_KEY',
    'q': 'Jimmy McMillan',
    'sort': '-first_file_date'
}
response = requests.get('https://api.open.fec.gov/v1/candidates/', params=params)
data = response.json()
data

{'api_version': '1.0',
 'results': [{'has_raised_funds': False,
   'incumbent_challenge': 'O',
   'federal_funds_flag': False,
   'party': 'REP',
   'incumbent_challenge_full': 'Open seat',
   'last_file_date': '2015-10-13',
   'last_f2_date': '2015-10-13',
   'inactive_election_years': None,
   'active_through': 2016,
   'election_districts': ['00'],
   'candidate_inactive': False,
   'party_full': 'REPUBLICAN PARTY',
   'cycles': [2016, 2018],
   'flags': 'P60016805',
   'election_years': [2016],
   'office_full': 'President',
   'first_file_date': '2015-10-13',
   'candidate_id': 'P60016805',
   'load_date': '2018-02-17T09:16:20+00:00',
   'district': '00',
   'state': 'US',
   'name': 'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH',
   'candidate_status': 'N',
   'district_number': 0,
   'office': 'P'},
  {'has_raised_funds': False,
   'incumbent_challenge': 'C',
   'federal_funds_flag': False,
   'party': 'REP',
   'incumbent_challenge_full': 'Challenger',
   'last_file_date': '2011-02-0

### Retrieving nested data

In [3]:
data['results'][0]['name']

'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH'

### [Homework 5](hw_5.ipynb)

As an in-class exercise, start with geocoding your own address.

### Reading into a DataFrame

In [4]:
params = {
    'api_key': 'DEMO_KEY',
    'has_raised_funds': 'true'
}
response = requests.get('https://api.open.fec.gov/v1/candidates/', params=params)
data = response.json()

pd.DataFrame(data['results'])

,name,last_file_date,district,candidate_id,candidate_status,incumbent_challenge_full,cycles,election_years,first_file_date,flags,...,district_number,party_full,load_date,active_through,office,candidate_inactive,has_raised_funds,inactive_election_years,party,office_full
0,"AADLAND, ERIK",2021-12-27,07,H2CO07170,N,Open seat,[2022],[2022],2021-12-27,H2CO07170,...,7,REPUBLICAN PARTY,2022-01-13T01:40:22+00:00,2022,H,False,True,None,REP,House
1,"AALDERS, TIM",2022-03-21,03,H2UT03280,C,Challenger,[2022],[2022],2020-03-24,H2UT03280,...,3,REPUBLICAN PARTY,2022-04-13T21:10:09+00:00,2022,H,False,True,None,REP,House
2,"AALDERS, TIMOTHY NOEL",2018-04-23,00,S2UT00229,P,Open seat,"[2012, 2014, 2016, 2018, 2020]","[2012, 2018]",2012-02-08,S2UT00229,...,0,CONSTITUTION PARTY,2019-03-27T16:02:41+00:00,2018,S,False,True,None,CON,Senate
3,"AALOORI, BANGAR REDDY",2019-10-17,22,H0TX22260,C,Open seat,[2020],[2020],2019-10-17,H0TX22260,...,22,REPUBLICAN PARTY,2020-03-18T21:13:37+00:00,2020,H,False,True,None,REP,House
4,"AAMODT, NORMAN O.",1978-07-05,16,H6PA16106,P,None,"[1976, 1978, 1980]","[1976, 1978]",1976-04-12,H6PA16106,...,16,REPUBLICAN PARTY,2002-03-30T00:00:00+00:00,1978,H,False,True,None,REP,House
5,"AANESTAD, SAMUEL",2012-02-22,01,H2CA01110,P,Challenger,"[2012, 2014, 2016]",[2012],2012-02-22,H2CA01110,...,1,REPUBLICAN PARTY,2013-04-26T09:04:30+00:00,2012,H,False,True,None,REP,House
6,"AARESTAD, DAVID",2017-04-26,06,H8CO06237,C,Challenger,[2018],[2018],2017-04-26,H8CO06237,...,6,DEMOCRATIC PARTY,2017-08-01T20:57:28+00:00,2018,H,False,True,None,DEM,House
7,"AARON, LAURA DAVIS",2007-03-13,00,P80002926,N,Open seat,"[2006, 2008, 2010, 2012, 2014, 2016]",[2008],2005-10-12,P80002926,...,0,DEMOCRATIC PARTY,2016-11-17T06:10:48+00:00,2008,P,False,True,None,DEM,President
8,"AAZAMI, SHERVIN",2022-01-24,32,H2CA30291,C,Challenger,[2022],[2022],2021-01-16,H2CA30291,...,32,DEMOCRATIC PARTY,2022-01-25T20:59:17+00:00,2022,H,False,True,None,DEM,House
9,"ABAIR, PETER JON",2000-02-02,01,H0MA01024,P,Challenger,"[2000, 2002, 2004]",[2000],2000-02-02,H0MA01024,...,1,REPUBLICAN PARTY,2002-04-12T00:00:00+00:00,2000,H,False,True,None,REP,House


## Back to 311 data

From [NYC Open Data Portal dataset page](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/data), click `Export` -> `SODA API` -> `API Docs`.

### Most open data sites have APIs

Often built on platforms that provide them, e.g.

- [NYC Open Data Portal](https://opendata.cityofnewyork.us/) built on [Socrata](https://dev.socrata.com/)
- [data.gov built on CKAN](https://www.data.gov/developers/apis)

### Example: 311 requests from the last week

In [5]:
from datetime import datetime, timedelta

now = datetime.utcnow()
now

datetime.datetime(2022, 4, 28, 17, 45, 40, 847519)

In [6]:
start = now - timedelta(weeks=1)
start

datetime.datetime(2022, 4, 21, 17, 45, 40, 847519)

In [7]:
start.isoformat()

'2022-04-21T17:45:40.847519'

Using the [Socrata query language (SoQL)](https://dev.socrata.com/docs/queries/):

In [8]:
data_id = 'erm2-nwe9'
params = {
    '$where': f"created_date between '{start.isoformat()}' and '{now.isoformat()}'"
}

url = f'https://data.cityofnewyork.us/resource/{data_id}.json'
response = requests.get(url, params=params)
data = response.json()

data

[{'unique_key': '53972143',
  'created_date': '2022-04-21T17:46:00.000',
  'agency': 'DEP',
  'agency_name': 'Department of Environmental Protection',
  'complaint_type': 'Noise',
  'descriptor': 'Noise: Alarms (NR3)',
  'incident_zip': '10460',
  'incident_address': '2123 BOSTON ROAD',
  'street_name': 'BOSTON ROAD',
  'cross_street_1': 'E 180 ST',
  'cross_street_2': 'BRONX PARK S',
  'address_type': 'ADDRESS',
  'city': 'BRONX',
  'facility_type': 'N/A',
  'status': 'Open',
  'community_board': '06 BRONX',
  'bbl': '2031380001',
  'borough': 'BRONX',
  'x_coordinate_state_plane': '1018046',
  'y_coordinate_state_plane': '246489',
  'open_data_channel_type': 'ONLINE',
  'park_facility_name': 'Unspecified',
  'park_borough': 'BRONX',
  'latitude': '40.843165857932014',
  'longitude': '-73.87785400273388',
  'location': {'latitude': '40.843165857932014',
   'longitude': '-73.87785400273388',
   'human_address': '{"address": "", "city": "", "state": "", "zip": ""}'}},
 {'unique_key': '5

In [9]:
pd.DataFrame(data)

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,cross_street_1,...,location_type,intersection_street_1,intersection_street_2,landmark,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,bridge_highway_segment,vehicle_type,due_date
0,53972143,2022-04-21T17:46:00.000,DEP,Department of Environmental Protection,Noise,Noise: Alarms (NR3),10460,2123 BOSTON ROAD,BOSTON ROAD,E 180 ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,53971049,2022-04-21T17:46:00.000,DEP,Department of Environmental Protection,Water System,Hydrant Leaking (WC1),10035,2195 3 AVENUE,3 AVENUE,E 119 ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,53972887,2022-04-21T17:46:05.000,NYPD,New York City Police Department,Abandoned Vehicle,With License Plate,11105,41-12 20 AVENUE,20 AVENUE,41 STREET,...,Street/Sidewalk,41 STREET,42 STREET,20 AVENUE,NaN,NaN,NaN,NaN,NaN,NaN
3,53968117,2022-04-21T17:46:23.000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,11213,234 SCHENECTADY AVENUE,SCHENECTADY AVENUE,STERLING PLACE,...,Street/Sidewalk,STERLING PLACE,ST JOHNS PLACE,SCHENECTADY AVENUE,NaN,NaN,NaN,NaN,NaN,NaN
4,53967862,2022-04-21T17:46:50.000,DOB,Department of Buildings,Building/Use,Illegal Conversion Of Residential Building/Space,NaN,42-30 79 STREET,79 STREET,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,53973599,2022-04-21T20:31:25.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,10466,655 EAST 230 STREET,EAST 230 STREET,CARPENTER AVENUE,...,Residential Building/House,CARPENTER AVENUE,LOWERRE PLACE,EAST 230 STREET,NaN,NaN,NaN,NaN,NaN,NaN
996,53969136,2022-04-21T20:31:36.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,10466,655 EAST 230 STREET,EAST 230 STREET,CARPENTER AVENUE,...,Residential Building/House,CARPENTER AVENUE,LOWERRE PLACE,EAST 230 STREET,NaN,NaN,NaN,NaN,NaN,NaN
997,53973495,2022-04-21T20:31:38.000,NYPD,New York City Police Department,Blocked Driveway,Partial Access,10461,1932 HOBART AVENUE,HOBART AVENUE,ST THERESA AVENUE,...,Street/Sidewalk,ST THERESA AVENUE,WILKINSON AVENUE,HOBART AVENUE,NaN,NaN,NaN,NaN,NaN,NaN
998,53967726,2022-04-21T20:32:00.000,DEP,Department of Environmental Protection,Water System,Hydrant Running Full (WA4),11214,NaN,NaN,NaN,...,NaN,28 AVENUE,CROPSEY AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Coincidence there were exactly 1,000 results?

### Pagination

- Most APIs limit the number of results returned.
- [Socrata defaults to 1,000.](https://dev.socrata.com/docs/queries/limit.html)
- Need to use a loop with parameters like [`$limit`](https://dev.socrata.com/docs/queries/limit.html)+[`$offset`](https://dev.socrata.com/docs/queries/offset.html) (Socrata) or `page`+`per_page` ([FEC](https://api.open.fec.gov/developers/))
   - [`append()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html) to DataFrame

## Things are going to differ by API

- Endpoints
- Supported parameters
- Response structure
   - [`json_normalize()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) can help
- Quality of documentation
- Helpfulness of errors
- Size/helpfulness of community

Gotta read and experiment.

## Homework 6

In real/ideal world, start with specific question and find data to answer it:

![project flow](extras/img/projectflow.png)

_Source: [Big Data and Social Science](https://textbook.coleridgeinitiative.org/chap-intro.html#the-structure-of-the-book)_

Data needed often doesn't exist or is hard (or impossible) to find/access

![project flow](extras/img/projectflow_amended.png)

[Homework 6](hw_6.ipynb)

## No homework/resubmissions will be accepted after Sunday 5/15 at 6:45pm ET

In other words, homework 6 cannot be late.

## Lecture 6